In [ ]:
!pip install ../input/pytorchcv/pytorchcv-0.0.55-py2.py3-none-any.whl --quiet

In [ ]:
import os
import sys
import cv2
import glob
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math

from pytorchcv.model_provider import get_model

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps
    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)       
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        sigmoid = torch.sigmoid(x)
        tanh_sp = torch.tanh(F.softplus(x)) 
        return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

class Mish(nn.Module):
    def forward(self, x):
        return MishFunction.apply(x)

def to_Mish(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, Mish())
        else:
            to_Mish(child)

class Head(torch.nn.Module):
  def __init__(self, in_f, out_f, hidden):
    super(Head, self).__init__()
    
    self.f = nn.Flatten()
    self.l = nn.Linear(in_f, hidden)
    self.m = Mish()
    self.d = nn.Dropout(0.75)
    self.o = nn.Linear(hidden, out_f)
    # self.o = nn.Linear(in_f, out_f)
    self.b1 = nn.BatchNorm1d(in_f)
    self.b2 = nn.BatchNorm1d(hidden)
    self.r = nn.ReLU()

  def forward(self, x):
    x = self.f(x)
    x = self.b1(x)
    x = self.d(x)

    x = self.l(x)
    x = self.r(x)
    x = self.b2(x)
    x = self.d(x)

    out = self.o(x)
    return out

class FCN(torch.nn.Module):
  def __init__(self, base, in_f, hidden):
    super(FCN, self).__init__()
    self.base = base
    self.h1 = Head(in_f, 1, hidden)
  
  def forward(self, x):
    x = self.base(x)
    return self.h1(x)

net = []

model = get_model("xception", pretrained=False)
model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer
model[0].final_block.pool = nn.Sequential(nn.AdaptiveAvgPool2d(1))
model = FCN(model, 2048, 512)
model = model.cuda()
model.load_state_dict(torch.load('../input/deepfake-models/model2.pth')) # .402
net.append(model)

# model = get_model("efficientnet_b1", pretrained=False)
# model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer
# model[0].final_pool = nn.Sequential(nn.AdaptiveAvgPool2d(1))
# model = FCN(model, 1280, 512)
# model = model.cuda()
# model.load_state_dict(torch.load('../input/deepfake-models/model (.251).pth')) # .41879
# net.append(model)

# model = get_model("xception", pretrained=False)
# model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer
# model[0].final_block.pool = nn.Sequential(nn.AdaptiveAvgPool2d(1))
# model = FCN(model, 2048, 512)
# model = model.cuda()
# model.load_state_dict(torch.load('../input/deepfake-models/model (.32555).pth')) # .439
# net.append(model)

model = get_model("efficientnet_b1", pretrained=False)
model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer
model[0].final_pool = nn.Sequential(nn.AdaptiveAvgPool2d(1))
model = FCN(model, 1280, 512)
model = model.cuda()
model.load_state_dict(torch.load('../input/deepfake-models/model (.259).pth')) # .392
net.append(model)

# model = get_model("efficientnet_b1", pretrained=False)
# model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer
# model[0].final_pool = nn.Sequential(nn.AdaptiveAvgPool2d(1))
# model = FCN(model, 1280, 256)
# model = model.cuda()
# model.load_state_dict(torch.load('../input/deepfake-models/model_17 (.2486).pth')) # .4099
# net.append(model)

model = get_model("efficientnet_b1", pretrained=False)
model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer
model[0].final_pool = nn.Sequential(nn.AdaptiveAvgPool2d(1))
model = FCN(model, 1280, 512)
model = model.cuda()
model.load_state_dict(torch.load('../input/deepfake-models/model_16 (.2755).pth')) # .40
net.append(model)

# model = get_model("efficientnet_b1", pretrained=False)
# model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer
# model[0].final_pool = nn.Sequential(nn.AdaptiveAvgPool2d(1))
# model = FCN(model, 1280, 512)
# model = model.cuda()
# model.load_state_dict(torch.load('../input/deepfake-models/model_20 (.243).pth')) # .410
# net.append(model)

In [ ]:
# from pytorchcv.model_provider import get_model
# # model = get_model("xception", pretrained=True)
# model = get_model("efficientnet_b1", pretrained=False)
# model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer
# model[0].final_pool = nn.AdaptiveAvgPool2d(1)
# # model[0].final_block.pool = nn.Sequential(nn.AdaptiveAvgPool2d(1))

# class LRCN(nn.Module):
#     def __init__(self, base, in_f, out_f):
#         super(LRCN, self).__init__()
#         self.cnn = base
        
#         self.LSTM = nn.LSTM(
#             input_size=in_f,
#             hidden_size=256,
#             num_layers=1,
#             batch_first=True
#         )

#         self.f1 = nn.Linear(256, 128)
#         self.f2 = nn.Linear(128, out_f)
#         self.r = nn.ReLU()
#         self.d = nn.Dropout(0.5)
        
#     def forward(self, x):
#         batch_size, timesteps, C, H, W = x.size()
#         x = x.view(batch_size * timesteps, C, H, W)
#         x = self.cnn(x)
#         x = x.view(batch_size, timesteps, -1)
#         self.LSTM.flatten_parameters()
#         x, (hn,hc) = self.LSTM(x)
#         x = self.d(self.r(self.f1(x[:,-1,:])))
#         x = self.f2(x)
#         return x

# model = LRCN(model, 1280, 1)
# model = model.cuda()
# model.load_state_dict(torch.load('../input/deepfake-models/lrcn (.337).pth'))
# net.append(model)

In [ ]:
from torchvision.transforms import Normalize

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
normalize_transform = Normalize(mean, std)

In [ ]:
import sys
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import time
import tensorflow as tf
import numpy as np
import glob
from tqdm import tqdm

import tensorflow as tf
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile('../input/mobilenet-face/frozen_inference_graph_face.pb', 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess=tf.compat.v1.Session(graph=detection_graph, config=config)
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    boxes_tensor = detection_graph.get_tensor_by_name('detection_boxes:0')    
    scores_tensor = detection_graph.get_tensor_by_name('detection_scores:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

def get_mobilenet_face(image):
    global boxes,scores,num_detections
    (im_height,im_width)=image.shape[:-1]
    imgs=np.array([image])
    (boxes, scores) = sess.run(
        [boxes_tensor, scores_tensor],
        feed_dict={image_tensor: imgs})
    max_=np.where(scores==scores.max())[0][0]
    box=boxes[0][max_]
    ymin, xmin, ymax, xmax = box
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
    left, right, top, bottom = int(left), int(right), int(top), int(bottom)
    return (left, right, top, bottom)
def crop_image(frame,bbox):
    left, right, top, bottom=bbox
    return frame[top:bottom,left:right]
def get_img(frame):
    return cv2.resize(crop_image(frame,get_mobilenet_face(frame)),(160,160))

def detect_video(video):
    capture = cv2.VideoCapture(video)
    v_len = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idxs = np.linspace(0,v_len,frame_count, endpoint=False, dtype=np.int)
    imgs=[]
    i=0
    for frame_idx in range(int(v_len)):
        ret = capture.grab()
        if not ret: 
            pass
        if frame_idx >= frame_idxs[i]:
            ret, frame = capture.retrieve()
            if not ret or frame is None:
                pass
            else:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                try:
                    face=get_img(frame)
                except Exception as err:
                    print(err)
                    continue
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                imgs.append(face)
            i += 1
            if i >= len(frame_idxs):
                break
    if len(imgs)<frame_count:
        return None
    return imgs

## Process all videos

In [ ]:
filenames = glob.glob('/kaggle/input/deepfake-detection-challenge/test_videos/*.mp4')
filenames.sort()

frame_count=30

num_faces = 0
probs = []
indexes = []
pbar = tqdm(filenames)
for filename in pbar:
    try:
        faces_ = detect_video(filename)
        if faces_ is None:
            probs.append(0.5)
#             break
            continue
            
        faces = []
        for face in faces_:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            img = np.rollaxis(cv2.resize(face, (150,150)), -1, 0)
            faces.append(img)
        
        faces2 = []
        for face in faces_:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            img = np.rollaxis(cv2.resize(face, (224,224)), -1, 0)
            faces2.append(img)
        
        faces = torch.from_numpy(np.array(faces)).float().cuda()
        faces2 = torch.from_numpy(np.array(faces2)).float().cuda()
        
        inputs = []
#         inputs.append(faces2)
#         normalize = [False]
        
#         inputs.append(faces)
#         inputs.append(faces)
#         inputs.append(faces)
#         inputs.append(faces2)
#         inputs.append(faces2)
#         inputs.append(faces2)
#         inputs.append(faces2)
#         normalize = [True,True,True,False,False,False,False]
        
        inputs.append(faces)
        inputs.append(faces2)
        inputs.append(faces2)
        normalize = [True,False,False]
        
        probs_2 = []
        
        # CNN
        with torch.no_grad():
            for j,model in enumerate(net):
                model.eval()
                probs_ = []
                for i,face in enumerate(inputs[j]):
                    if normalize[j]:
                        face = normalize_transform(face / 255.)
                    out = model(face[None])
                    out = torch.sigmoid(out.squeeze())
                    probs_.append(out.item())
                probs_2.append(np.mean(np.array(probs_)))
        
        # LRCN
#         faces_ = []
#         for face in faces:
#             face = normalize_transform(face / 255.)
#             faces_.append(face)
#         faces_ = torch.stack(faces_, dim=0).cuda().float()
        
#         with torch.no_grad():
#             for model in net:
#                 model.eval()
#                 out = model(faces_[None])
#                 out = torch.sigmoid(out.squeeze())
#                 probs_2.append(out.item())

        mult = 1
        for prob in probs_2:
            mult *= prob
        probs.append(mult**(1/float(len(net))))
        
        num_faces += len(faces)
        pbar.set_description(f'Faces found: {num_faces}')
#         break
    except:
        probs.append(0.5)
#         break

In [ ]:
probs = np.asarray(probs)
probs[probs!=probs] = 0.5 # Remove NaNs
# probs = np.clip(probs, 0.01, 0.99)

plt.hist(probs, 40)

filenames = [os.path.basename(f) for f in filenames]

submission = pd.DataFrame({'filename': filenames, 'label': probs})
submission.to_csv('submission.csv', index=False)
submission